In [ ]:
import pandas as pd
data = pd.read_csv("c:\\Users\\owner\\Downloads\\week1.csv",
                   index_col=0)#,
#                   parse_dates=[1])
pd.to_datetime(data.ReceivedDateTime)
pd.get_dummies( data.passorfail ) #One Hot Encoding

In [ ]:
import numpy as np
data_num = data.select_dtypes(include="float")
data_cat = data.select_dtypes(exclude="float")

data_num.describe()
np.where(data_num.SpindleSpeed_mean >1188, 1, 0)

def func1(x):
    if x>1188:
        return 1
    else:
        return 0
data_num.SpindleSpeed_mean.apply(func1)     

In [ ]:
data2 = pd.concat( [data_num, data_cat], axis=1 )

pd.to_datetime(data.ReceivedDateTime)
pd.get_dummies( data.passorfail ) #One Hot Encoding

from sklearn.model_selection import train_test_split
train, test = train_test_split( data2, 
                               stratify=data2.passorfail )


In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE
X, y = SMOTE(sampling_strategy = 0.2 , # 
             k_neighbors = 5,
             random_state=1).fit_resample(data_num, 
                                           data.passorfail)
print( data_num.shape, data.passorfail.shape )
print( X.shape, y.shape, y.value_counts())     

(1085, 40) (1085,)
(1183, 40) (1183,) passorfail
0    986
1    197
Name: count, dtype: int64


In [ ]:
files = os.listdir('data2')
for i in files:
    if i.endswith('.csv'):
        print(i)
[ i for i in os.listdir('data2') if i.endswith('.csv')]

In [ ]:
os.makedirs("roy")

In [ ]:
#data2 폴더 밑 data_politics의 파일들 중에서,
#ROY라는 사람의 발언만 모아서 data2/data_politics/roy폴더에 
#옮기세요
files3 = [ i for i in os.listdir("data_politics") 
 if i.endswith("_ROY.txt")]
for i in files3:
    shutil.move("data_politics\\"+i, 
                "roy\\"+i)

In [ ]:
os.chdir("keyword_data")

In [ ]:
#keyword_data 폴더 밑의 폴더들의 파일들을 
#하나의 데이터프레임으로 합쳐보세요
import pandas as pd
result = pd.DataFrame()
for i in ['1', '2', '3']:
  for j in ['1', '2', '3']:
    for q in ['1', '2']:
      tmp = 'k'+i+'_'+j+'_'+q
      for t in os.listdir(tmp):
        file = tmp+"\\"+t
        if os.path.getsize(file) > 0:
          df_tmp = pd.read_csv(tmp+"\\"+t, header=None)
          result = pd.concat([result, df_tmp])


In [ ]:
#agv.zip 다운로드->하나의 데이터프레임
# -> imputing->scaling->OHE->파퀘 저장, #폴더값이 Y가 되도록 지정해주세요.
#pd.read_csv(    , skiprows= )
os.chdir("c:\\users\\owner\\agv")
folders = os.listdir()
result = pd.DataFrame()
for cat in folders: 
 for i in os.listdir(cat):  
  tmp = pd.read_csv(cat+"\\"+i, skiprows=9, header=None )
  tmp["label"] = cat 
  result = pd.concat( [result, tmp] ) 
result.columns = ["v1", "v2", "v3","v4","v5","label"]
result = result.drop("v5", axis=1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform( result.iloc[:, 0:4] )
X = pd.DataFrame( scaled, columns=["v1", "v2", "v3", "v4"])
y = pd.get_dummies(result["label"])
y.index=X.index
result2 = pd.concat( [X,y], axis=1, ignore_index=True)
result2.to_parquet("agv.parquet")  


In [ ]:
import os
os.getcwd()
os.chdir("c:\\Users\\owner\\images2")


In [ ]:
from PIL import Image
import numpy as np

X = []
Y = []
for cat in ["camera", "chair"]: 
 for i in os.listdir(cat):  
    img = Image.open(cat+"\\"+i)
    img = img.convert("RGB")
    img = img.resize((30, 30))
    data=np.array(img)
    X.append(data)
    Y.append(cat)

X=np.array(X)
Y=np.array(Y)

X = X.reshape( 112, 2700)  #4차원->2차원
X = X/255  #minmaxscaling
Y = pd.get_dummies(Y) #OHE
#partition
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X,Y)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit( X_train, y_train)

